In [1]:
#https://scrapfly.io/blog/web-scraping-with-selenium-and-python/
# https://stackoverflow.com/questions/53475578/python-selenium-multiprocessing

TODO
- явно есть ошибки не корретного парсинга, можно найти кейсы по дешевым билетам
- подумать может сделать без селениума
- на сервер запустить скрипт

In [2]:

import os
import json
import asyncio
from concurrent.futures.thread import ThreadPoolExecutor

import nest_asyncio
nest_asyncio.apply()

In [3]:
from parsel import Selector
from itertools import product

from tqdm import tqdm_notebook as tqdm

import glob
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

add_zero = lambda x: '0' + str(x) if len(str(x)) < 2 else str(x)

options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
                    "prefs", {"profile.managed_default_content_settings.images": 2}
                        )
service = Service()

In [4]:
def parse_page(page):
    soup = BeautifulSoup(page, 'lxml')

    prices = soup.find_all('div', attrs={'class':'ticket-desktop__side-container'})

    price_ticket = list(map(lambda x: x.find('span', attrs={'data-test-id':'price'}), prices))
    price_ticket = list(map(lambda x: x.text.replace('\u202f','').replace('\u2009₽',''), price_ticket))
    
    price_luggage = list(map(lambda x: x.find('div', text='Багаж'), prices))
    price_luggage = list(map(lambda x: x.find_next('span').get_text(strip=True) if x else '0',  price_luggage))  
    del_words = ['+\u2009','\u202f', '\u2009₽', '₽']
    for word in del_words:
        price_luggage = list(map(lambda x: x.replace(word,'') if x else '0', price_luggage))
    
    wo_luggage = list(map(lambda x: 1 if x.find('div', text='Без багажа') else 0, prices))

    duration = soup.find_all('div', attrs={'class':'segment-route__duration'})
    duration = list(map(lambda x: x.text.replace('В пути:  ',''), duration))

    transfer = soup.find_all('div', attrs={'class':'segment-route__path'})
    transfer = list(map(lambda x: x.find_all('div', attrs={'class':'segment-route__stop'}), transfer))
    transfer = list(map(lambda x: list(map(lambda y: y['data-iatas'], x)), transfer))

    number_transfer = list(map(lambda x: len(x), transfer))
    transfer = list(map(lambda x: ', '.join(x), transfer))
    transfer = list(map(lambda x: x.replace('\n',"->"), transfer))

    keys = ['price', 'luggage', 'without_luggage', 'duration', 'transfer', 'number_transfer']

    zipped = zip(price_ticket, price_luggage, wo_luggage, duration, transfer, number_transfer)

    data = [dict(zip(keys, values)) for values in zipped]
    return data

In [5]:
def list_dates(datetime_start, datetime_end):
    delta = timedelta(days = 1)
    range_dates = []
    date = datetime_start
    while date <= datetime_end:
        range_dates.append(date)
        date = date + delta   
    return range_dates

In [6]:
def get_ticket(origin, destination, date, time_sleep = 20):
    root = 'tickets'
    driver = webdriver.Chrome(options=options, chrome_options=chrome_options, service = service)
    month, day = add_zero(date.month), add_zero(date.day)
    date = date.strftime('%Y-%m-%d')
    url = 'https://www.aviasales.ru/search/' + origin + day + month + destination + str(number_ticket)
    driver.get(url)
    time.sleep(time_sleep)
    try:
        element = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@id='ui-datepicker-div']//a[@class='ui-state-default']")))
    except TimeoutException:
        pass
    page = driver.page_source
    tickets = parse_page(page)
    tickets = [dict(item, **{'date':date, 'origin':origin, 'destination':destination}) for item in tickets]
    if not os.path.isdir(root):
        os.makedirs(os.path.join(root))
        os.makedirs(os.path.join(root, 'json'))
        os.makedirs(os.path.join(root, 'screenshots'))
        
    with open(os.path.join(root, 'json', origin + '_' + destination + '_' + date + '.json'), 'w', encoding='utf-8') as f:
        json.dump(tickets, f)
    
    driver.set_window_size(1920, 1080)
    time.sleep(2)
    driver.save_screenshot(os.path.join(root, 'screenshots', origin + '_' + destination + '_' + date + '.png'))
    driver.close()
            
def scrape(origin, destination, datetime_object, *, loop):
    loop.run_in_executor(executor, get_ticket, origin, destination, datetime_object)

In [7]:
executor = ThreadPoolExecutor(1)
loop = asyncio.get_event_loop()


origins =  ['BUD', 'TSR', 'OSI', 'PEV', 'SOB', 'DEB', 'ZAG'] #['LED', 'MOW', 'KZN']
destinations = ['HEL']# ['BEG', 'TGD', 'TIA', 'TIV']
range_date = ['2023-10-20', '2023-11-10']
number_ticket = 1

date_start, date_end = range_date[0], range_date[1]
datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')
range_dates = list_dates(datetime_start, datetime_end)

parameters = list(product(origins, destinations, list_dates(datetime_start, datetime_end)))

# билеты туда
for origin, destination, datetime_object in tqdm(parameters, total=len(parameters), desc='parse ticket'):
    scrape(origin, destination, datetime_object, loop=loop)

# билеты обратно
# for destination, origin, datetime_object in tqdm(parameters, total=len(parameters), desc='parse back ticket'):
#     scrape(origin, destination, datetime_object, loop=loop)


parse ticket:   0%|          | 0/154 [00:00<?, ?it/s]

In [8]:
# чтобы обработать все json
root = '.\\tickets\\json'
json_pattern = os.path.join(root,'*.json')
file_list = glob.glob(json_pattern)

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_json(file) # read data frame from json file
    dfs.append(data) # append the data frame to the list

df = pd.concat(dfs, ignore_index=True) 
df.to_csv('tickets.txt', sep = ';', index = False, encoding='utf-8-sig')

ValueError: No objects to concatenate

Future exception was never retrieved
future: <Future finished exception=AttributeError("'NoneType' object has no attribute 'find_next'")>
Traceback (most recent call last):
  File "C:\Users\ilyachulkov\Anaconda3\lib\concurrent\futures\thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_19152\4235567714.py", line 15, in get_ticket
    tickets = parse_page(page)
  File "C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_19152\973094573.py", line 10, in parse_page
    price_luggage = list(map(lambda x: x.find('div', text='Багаж').find_next('span').get_text(strip=True), prices))
  File "C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_19152\973094573.py", line 10, in <lambda>
    price_luggage = list(map(lambda x: x.find('div', text='Багаж').find_next('span').get_text(strip=True), prices))
AttributeError: 'NoneType' object has no attribute 'find_next'
Future exception was never retrieved
future: <Future finis

Future exception was never retrieved
future: <Future finished exception=AttributeError("'NoneType' object has no attribute 'find_next'")>
Traceback (most recent call last):
  File "C:\Users\ilyachulkov\Anaconda3\lib\concurrent\futures\thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_19152\4235567714.py", line 15, in get_ticket
    tickets = parse_page(page)
  File "C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_19152\973094573.py", line 10, in parse_page
    price_luggage = list(map(lambda x: x.find('div', text='Багаж').find_next('span').get_text(strip=True), prices))
  File "C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_19152\973094573.py", line 10, in <lambda>
    price_luggage = list(map(lambda x: x.find('div', text='Багаж').find_next('span').get_text(strip=True), prices))
AttributeError: 'NoneType' object has no attribute 'find_next'
